In [1]:
import os 



In [2]:
%pwd

'/media/puzan/NewVolume/MyFolder/research'

In [3]:
os.chdir("../")

In [4]:
os.getcwd()

'/media/puzan/NewVolume/MyFolder'

In [5]:
#create entity that is return type
from dataclasses import dataclass
from pathlib import Path
#data class bhaneko class banauxa instance haru

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir=Path

In [6]:
%pwd

'/media/puzan/NewVolume/MyFolder'

In [18]:
from translator.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from translator.utils.common import read_yaml,create_directories


In [19]:
print(CONFIG_FILE_PATH,PARAMS_FILE_PATH)

config/config.yaml param/params.yaml


In [20]:
with open(CONFIG_FILE_PATH,"r") as f:
    print(f.readlines())

['artifacts_root: artifacts\n', '\n', 'data_ingestion:\n', '  root_dir: artifacts/data_ingestion\n', '  source_URL: https://github.com/Puzan789/Datas/blob/main/translatordata.zip\n', '  local_data_file: artifacts/data_ingestion/data.zip\n', '  unzip_dir: artifacts/data_ingestion\n']


In [ ]:
# configuration manager

In [22]:
class ConfigurationManager:
    def __init__(self,config_file_path:CONFIG_FILE_PATH,params_file_path:PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([Path(config.root_dir)])
        data_ingestion_config=DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

